# Automation of file export

**Importing and formating the data**

In [33]:
import pandas as pd
from datetime import datetime
df = pd.read_csv('C:/Users/Masha/Downloads/lesson_2_data.csv', 
            encoding = 'windows-1251', sep = ';')

**Checking the columns names**

In [34]:
df.columns

Index(['Номер', 'Дата создания', 'Дата оплаты', 'Title', 'Статус',
       'Заработано', 'Город', 'Платежная система'],
      dtype='object')

**Translating the columns names to English**

In [35]:
df=df.rename(columns={'Номер': 'number', 
                   'Дата создания': 'create_date', 
                   'Дата оплаты': 'payment_date', 
                   'Title': 'title', 
                   'Статус': 'status',
                   'Заработано': 'money', 
                   'Город': 'city', 
                   'Платежная система': 'payment_method'})

df.head()

,number,create_date,payment_date,title,status,money,city,payment_method
0,1062823,01.12.2019 10:50,01.12.2019 10:52,Курс обучения «Эксперт»,Завершен,29597.5,Чита,Сбербанк эквайринг
1,1062855,01.12.2019 20:53,01.12.2019 21:27,Курс обучения «Эксперт»,Завершен,17450.3,Краснодар,Яндекс.Касса
2,1062856,01.12.2019 21:43,NaN,Курс обучения «Специалист»,Отменен,0.0,NaN,NaN
3,1062880,03.12.2019 0:18,NaN,Курс обучения «Консультант»,Отменен,0.0,г.Москва и Московская область,NaN
4,1062899,03.12.2019 21:43,NaN,Курс обучения «Эксперт»,Отменен,0.0,г.Москва и Московская область,NaN


**Creating a variable for self-checking after finishing the analysis**

In [36]:
all_money = df.money.sum()
all_money

992103.5900000001

**Calculating the profit and orders per course**

In [37]:
money_by_title = df.query("status == 'Завершен'") \
    .groupby (['title'],as_index = False) \
    .aggregate ({'money' : 'sum', 'number' : 'count'}) \
    .sort_values('money', ascending = False) \
    .rename(columns={'number': 'success_orders'})

money_by_title

,title,money,success_orders
5,Подписка «ОНЛАЙН ДИЕТОЛОГ» с ежемесячным автос...,366947.20,125
0,Курс обучения «Консультант»,208163.49,31
1,Курс обучения «Специалист»,160862.64,7
2,Курс обучения «Эксперт»,148992.80,5
4,Курс от Школы Диетологов. Повышение квалификац...,88384.92,9
3,Курс от Школы Диетологов. Бизнес,18752.54,3


**Calculating the profit and orders per course and per city**

In [38]:
money_by_city = (df.query("status == 'Завершен'")
    .groupby (['title', 'city'],as_index = False)
    .agg ({'money' : 'sum', 'number' : 'count'})
    .sort_values('money', ascending = False)
    .rename(columns={'number': 'success_orders'}) )

money_by_city.head()

,title,city,money,success_orders
32,Курс обучения «Эксперт»,г.Санкт-Петербург и Ленинградская область,59195.00,2
107,Подписка «ОНЛАЙН ДИЕТОЛОГ» с ежемесячным автос...,г.Москва и Московская область,46967.04,16
29,Курс обучения «Эксперт»,Балхаш,42750.00,1
58,Подписка «ОНЛАЙН ДИЕТОЛОГ» с ежемесячным автос...,Краснодарский край,38169.78,13
23,Курс обучения «Специалист»,Красноярский край,29695.70,1


**Self-checking and printing the file automatically**

In [39]:
# get the today's date
today_day = datetime.today().strftime('%Y-%m-%d')

# create a file name that automatially updates the date
file_name = 'money_title_{}.csv'
file_name = file_name.format(today_day)

In [40]:
if int(money_by_title.money.sum())==int(all_money):
    print('OK: File {} is written.'.format(file_name))
    money_by_title.to_csv(file_name, index=False)
    
else:
    print('ERROR!')

OK: File money_title_2022-05-16.csv is written.
